In [1]:
import requests
import csv
import time
from bs4 import BeautifulSoup

# Danh sách công ty và mã tương ứng
companies = {
    "Hoàng Anh Gia Lai": "hag",
    "Vinamilk": "vnm",
    "Hòa Phát": "hpg",
    "FPT": "fpt",
    "MB Bank": "mbb"
}

years = list(range(2005, 2025))

# Hàm tải nội dung với retry
def get_with_retry(url, max_retries=5, wait=3):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36"
    }

    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            print(f"    ✅ Tải thành công sau {attempt} lần thử")
            return response
        except requests.exceptions.HTTPError as e:
            print(f"    ⚠️ Lỗi HTTP {response.status_code} - thử lại sau {wait} giây (lần {attempt})")
        except requests.exceptions.RequestException as e:
            print(f"    ❌ Lỗi kết nối: {e} - thử lại sau {wait} giây (lần {attempt})")

        time.sleep(wait)

    print("    ❌ Không thể tải trang sau nhiều lần thử.")
    return None

# Bắt đầu xử lý từng công ty
for company_name, ticker in companies.items():
    print(f"🔍 Đang xử lý công ty: {company_name} ({ticker})")

    final_table = []

    for year_index, year in enumerate(years):
        url = f'https://cafef.vn/du-lieu/bao-cao-tai-chinh/{ticker}/bsheet/{year}/4/0/0/bao-cao-tai-chinh-.chn'
        print(f"  -> Năm {year}")

        response = get_with_retry(url)
        if response is None:
            continue  # bỏ qua năm này nếu không tải được

        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', id='tableContent')

        if not table:
            print(f"    ❌ Không tìm thấy bảng dữ liệu")
            continue

        rows = table.find_all('tr')
        current_year_data = []

        for i, row in enumerate(rows):
            cols = row.find_all('td')
            if not cols:
                continue

            if len(cols) > 8:
                cols = cols[:-8]
            else:
                continue

            row_data = [col.text.strip() for col in cols]

            if len(row_data) < 2:
                continue

            if i == 0:
                if year_index == 0:
                    for q in range(1, len(row_data)):
                        if row_data[q] == '':
                            row_data[q] = f"Quý {q} năm {year}"
                    final_table.append(row_data)
                else:
                    new_titles = [f"Quý {q} năm {year}" for q in range(1, len(row_data))]
                    final_table[0].extend(new_titles)
            else:
                if year_index == 0:
                    current_year_data.append(row_data)
                else:
                    current_year_data.append(row_data[1:])  # bỏ cột đầu (chỉ tiêu)

        if year_index == 0:
            final_table.extend(current_year_data)
        else:
            for i in range(1, len(final_table)):
                final_table[i].extend(current_year_data[i - 1])

    # Ghi ra file CSV
    output_file = f"report 1/Tổng hợp báo cáo {company_name}.csv"
    with open(output_file, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(final_table)

    print(f"✅ Đã lưu vào file: {output_file}\n")


🔍 Đang xử lý công ty: Hoàng Anh Gia Lai (hag)
  -> Năm 2005
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2006
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2007
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2008
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2009
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2010
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2011
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2012
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2013
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2014
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2015
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2016
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2017
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2018
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2019
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2020
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2021
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2022
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2023
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2024
    ✅ Tải